# TP SBs to observe

In [1]:
# The first line is only for the IPython Notebook interface
# If you are working in the ipython console replace by `%matplotlib`
%matplotlib inline

import DataBase as db
import pandas as pd
import numpy as np
import datetime as dt
import ephem

import matplotlib.pyplot as plt
#import seaborn as sns

import cx_Oracle
import os

import cycle_tools as ct
#import plotly.plotly as py

## Load APA database, process SBs and create summary Dataframe

In [2]:
datas = db.Database(verbose=False, forcenew=False)

In [3]:
datas.process_sbs(forcenew=True)
datas.do_summarize_sb()

Processing Phase II SBs ....
Warning, uid://A002/X6ac013/X1 is corrupt

Warning, uid://A002/X6ac013/X2 is corrupt

Warning, uid://A002/X788a57/X37 is corrupt

Warning, uid://A002/X788a57/X38 is corrupt
.....
Warning, uid://A001/X121/X300 is corrupt
 Done!
Processing Phase I SBs .........  Done!
42


In [37]:
tp = datas.summary_sb.query('array == "TP-Array" and PRJ_LETTER_GRADE != "C" and SB_STATE in ["Ready","FullyObserved","Suspended", "Waiting"]')

In [38]:
what=pd.merge(tp, datas.fieldsource[['SB_UID', 'name', 'RA', 'DEC', 'solarSystem']], how='left', on='SB_UID')

In [39]:
what.name.unique()

array(['Pointing Template (Cal Group)',
       'Pointing Template (Science Group)', 'Amplitude', 'Delay',
       'Primary:', 'Focus', '3c454.3', 'calibrator_J1256'], dtype=object)

In [40]:
# Filter SG with at least 2 TP SBs (otherwise it means they don't have an Amplitude SB)
tp_sg = tp.groupby('SG_ID').SB_UID.count()[tp.groupby('SG_ID').SB_UID.count() > 1].index.unique()

In [41]:
TPlist = pd.merge(tp.query('SG_ID in @tp_sg'), datas.fieldsource.query('name in ["Amplitude", "3c454.3"]')[['SB_UID', 'name', 'RA', 'DEC', 'solarSystem']], on='SB_UID', how='left')[['CODE', 'PRJ_LETTER_GRADE', 'SB_UID', 'SG_ID', 'sbName', 'band', 'repfreq', 'RA_x', 'DEC_x', 'execount', 'observed', 'name', 'RA_y', 'DEC_y', 'solarSystem', 'SB_STATE']].copy()

In [42]:
datas.summary_sb.query('CODE == "2013.1.00556.S"')

,OBSPROJECT_UID,CODE,PRJ_LETTER_GRADE,PRJ_STATUS,isCycle2,SB_UID,SG_ID,OUS_ID,sbName,sbStatusXml,repfreq,band,array,RA,DEC,minAR_ot,maxAR_ot,execount,isPolarization,maxPWVC,array12mType,AR,LAS,minArrayAR,maxArrayAR,AR100GHz,LAS100GHz,minArrayAR100GHz,maxArrayAR100GHz,C34_1,C34_2,C34_3,C34_4,C34_5,C34_6,C34_7,allowed12m,corr_allowed12m,phase,SB_ETC_exec,observed,SB_ETC_total,SB_ETC_remain,LST,bestconf,SB_STATE
2256,uid://A001/X10e/X4d7,2013.1.00556.S,B,InProgress,True,uid://A001/X197/X9,uid://A001/X10e/X4d7_1,X1195710557,LMC-N11B_a_03_TE,Canceled,115.1876,ALMA_RB_03,TWELVE-M,74.22369,-66.40623,1.59141,1.945057,1,False,5.1,Ext,2,45,1.4,2.2,1.984651,44.654647,1.389256,2.183116,0,1,1,0,0,0,0,2,2,II,1.042509,0,1.042509,1.042509,4,C34-2,Canceled
2257,uid://A001/X10e/X4d7,2013.1.00556.S,B,InProgress,True,uid://A001/X197/Xa,uid://A001/X10e/X4d7_1,X1195710557,LMC-N11B_a_03_7M,Canceled,115.1876,ALMA_RB_03,SEVEN-M,74.22369,-66.40623,0,0,3,False,5.1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,II,0.695006,0,2.085017,2.085017,4,None,Canceled
2258,uid://A001/X10e/X4d7,2013.1.00556.S,B,InProgress,True,uid://A001/X197/Xb,uid://A001/X10e/X4d7_1,X1195710557,3c454.3_115.3_a_03_TP,Waiting,115.1876,ALMA_RB_03,TP-Array,343.4906,16.14821,0,0,1,False,5.1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,II,0.320772,1,0.320772,0.000000,22,None,FullyObserved
2259,uid://A001/X10e/X4d7,2013.1.00556.S,B,InProgress,True,uid://A001/X197/Xc,uid://A001/X10e/X4d7_1,X1195710557,LMC-N11B_115.3_a_03_TP,Waiting,115.1876,ALMA_RB_03,TP-Array,74.22369,-66.40623,0,0,12,False,5.1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,II,0.320772,2,3.849263,3.207719,4,None,Ready


In [43]:
TPlist.RA_x = TPlist.RA_x / 15.
TPlist.RA_y = TPlist.RA_y / 15

In [44]:
nota = TPlist.query('name == "Amplitude"').SG_ID.unique()

In [45]:
# These SBs have an amplitude calibrator that hasn't "Amplitude" as name
TPlist.query('SG_ID not in @nota')

,CODE,PRJ_LETTER_GRADE,SB_UID,SG_ID,sbName,band,repfreq,RA_x,DEC_x,execount,observed,name,RA_y,DEC_y,solarSystem,SB_STATE
12,2012.1.00080.S,B,uid://A002/X8981ca/X1,uid://A001/Xa0/X67_1,3c454.3_b_03_TP,ALMA_RB_03,97.981,22.89937,16.14821,1,9,3c454.3,22.89937,16.14821,Uranus,Ready
13,2012.1.00080.S,B,uid://A002/X8981ca/X2,uid://A001/Xa0/X67_1,GC50MC_b_03_TP,ALMA_RB_03,97.981,17.76429,-28.98907,24,0,NaN,NaN,NaN,NaN,Ready
169,2013.1.01114.S,B,uid://A001/X147/X9d,uid://A001/X112/X92_1,J1256_a_03_TP,ALMA_RB_03,97.99631,12.93644,-5.789312,1,5,NaN,NaN,NaN,NaN,FullyObserved
170,2013.1.01114.S,B,uid://A001/X147/X9e,uid://A001/X112/X92_1,Orion_In_a_03_TP,ALMA_RB_03,97.99631,5.590238,-5.163446,10,10,NaN,NaN,NaN,NaN,FullyObserved


In [46]:
TPlist.solarSystem.unique()

array(['Uranus', nan, 'Unspecified'], dtype=object)

In [47]:
tm_amplitude_ready = TPlist.query('(solarSystem != "Uranus" or RA_y != 0) and (name in ["Amplitude", "3c454.3"])').copy()

In [48]:
tm_amplitude_ready.sort(['RA_y', 'CODE'])

,CODE,PRJ_LETTER_GRADE,SB_UID,SG_ID,sbName,band,repfreq,RA_x,DEC_x,execount,observed,name,RA_y,DEC_y,solarSystem,SB_STATE
94,2012.1.00394.S,B,uid://A002/X9a055b/X2c,uid://A001/Xa0/X875_1,J0319+4130_f_03_TP,ALMA_RB_03,88.63039,0,0,1,0,Amplitude,3.330044,41.51169,Unspecified,Waiting
99,2012.1.00394.S,B,uid://A002/X9a055b/X31,uid://A001/Xa0/X875_2,J0319+4130_j_03_TP,ALMA_RB_03,93.17213,0,0,1,0,Amplitude,3.330044,41.51169,Unspecified,Waiting
106,2012.1.00394.S,B,uid://A002/X9a055b/X38,uid://A001/Xa0/X875_4,J1256-0547_b_03_TP,ALMA_RB_03,93.17203,0,0,1,1,Amplitude,12.93644,-5.789312,Unspecified,FullyObserved
143,2012.1.00538.S,B,uid://A002/X9a055b/X53,uid://A001/Xa0/Xc33_3,3c279_89_c_03_TP,ALMA_RB_03,89.19936,12.93644,-5.789312,1,1,Amplitude,12.93644,-5.789313,Unspecified,Ready
36,2012.1.00001.S,B,uid://A001/X147/X34b,uid://A001/X85/X1_1,3c279_114.8_a_03_TP,ALMA_RB_03,114.79,12.93644,-5.789313,1,2,Amplitude,12.93644,-5.789313,Unspecified,FullyObserved
58,2012.1.00382.S,B,uid://A002/X996c88/X1c,uid://A001/Xa0/X825_1,3C279_b_03_TP,ALMA_RB_03,114.7,12.93644,-5.789312,1,9,Amplitude,12.93644,-5.789313,Unspecified,FullyObserved
60,2012.1.00382.S,B,uid://A002/X996c88/X1e,uid://A001/Xa0/X825_2,3C279_c_03_TP,ALMA_RB_03,110.1994,12.93644,-5.789313,1,3,Amplitude,12.93644,-5.789313,Unspecified,FullyObserved
104,2012.1.00394.S,B,uid://A002/X9a055b/X36,uid://A001/Xa0/X875_3,J1256-0547_a_03_TP,ALMA_RB_03,88.6303,0,0,5,2,Amplitude,12.93644,-5.789313,Unspecified,Suspended
54,2012.1.00501.S,B,uid://A001/X196/X3f,uid://A001/Xa0/Xb45_3,3c279_114.6_b_03_TP,ALMA_RB_03,114.5625,12.93644,-5.789313,10,3,Amplitude,12.93644,-5.789313,Unspecified,FullyObserved
139,2012.1.00538.S,B,uid://A002/X9a055b/X4f,uid://A001/Xa0/Xc33_1,3c279_93_a_03_TP,ALMA_RB_03,93.18471,12.93644,-5.789312,10,2,Amplitude,12.93644,-5.789313,Unspecified,FullyObserved


In [49]:
print tm_amplitude_ready.RA_y.unique()

[22.899374426666668 12.936435166666666 22.899374416666667
 22.899374444444444 3.3300444444444444 12.936435155555555]


In [53]:
ok_block19_sg = tm_amplitude_ready.query('8 < RA_y < 19').SG_ID.unique()
ok_block19_ampSb = tm_amplitude_ready.query('8 < RA_y < 19').SB_UID.unique()
ok_block19_scSb = TPlist.query('SG_ID in @ok_block19_sg and SB_UID not in @ok_block19_ampSb and SB_STATE != "FullyObserved"')
ok_block19_scSb[['CODE', 'PRJ_LETTER_GRADE', 'SB_UID', 'sbName', 'band', 'RA_x', 'DEC_x', 'execount', 'observed', 'SB_STATE']]

,CODE,PRJ_LETTER_GRADE,SB_UID,sbName,band,RA_x,DEC_x,execount,observed,SB_STATE
125,2012.1.00641.S,B,uid://A001/X147/Xd8,cloud_22_b_03_TP,ALMA_RB_03,5.788889,-70.69167,2,0,Waiting
127,2012.1.00641.S,B,uid://A001/X147/Xda,cloud_2_b_03_TP,ALMA_RB_03,4.793056,-67.21667,2,0,Waiting
129,2012.1.00641.S,B,uid://A001/X147/Xdc,cloud_55_b_03_TP,ALMA_RB_03,5.129167,-68.99167,2,0,Waiting
131,2012.1.00641.S,B,uid://A001/X147/Xde,cloud_21_b_03_TP,ALMA_RB_03,5.743056,-69.43333,2,0,Waiting
133,2012.1.00641.S,B,uid://A001/X147/Xe0,cloud_19_b_03_TP,ALMA_RB_03,5.663889,-70.11667,2,0,Waiting
142,2012.1.00538.S,B,uid://A002/X9a055b/X52,G305_98_b_03_TP,ALMA_RB_03,13.17825,-62.72097,1,1,Ready
144,2012.1.00538.S,B,uid://A002/X9a055b/X54,G305_89_c_03_TP,ALMA_RB_03,13.17825,-62.72097,1,1,Ready


In [57]:
ok_block19end_sg = tm_amplitude_ready.query('RA_y > 22').SG_ID.unique()
ok_block19end_ampSb = tm_amplitude_ready.query('RA_y > 22').SB_UID.unique()
ok_block19end_scSb = TPlist.query('SG_ID in @ok_block19end_sg and SB_UID not in @ok_block19end_ampSb and SB_STATE != "FullyObserved"')
ok_block19end_scSb[['CODE', 'PRJ_LETTER_GRADE', 'SB_UID', 'sbName', 'band', 'RA_x', 'DEC_x', 'execount', 'observed', 'SB_STATE']]

,CODE,PRJ_LETTER_GRADE,SB_UID,sbName,band,RA_x,DEC_x,execount,observed,SB_STATE
13,2012.1.00080.S,B,uid://A002/X8981ca/X2,GC50MC_b_03_TP,ALMA_RB_03,17.76429,-28.98907,24,0,Ready
22,2013.1.00269.S,B,uid://A001/X147/X8f,SgrB2_a_03_TP,ALMA_RB_03,17.78873,-28.39161,40,36,Ready
33,2013.1.00351.S,B,uid://A002/X996c88/X76,NGC300_a_03_TP,ALMA_RB_03,0.9148553,-37.68438,13,0,Ready
35,2013.1.00351.S,B,uid://A002/X996c88/X78,NGC300_b_03_TP,ALMA_RB_03,0.9148553,-37.68438,13,0,Ready
65,2013.1.00532.S,B,uid://A002/X9908b7/X3f,ngc628_a_03_TP,ALMA_RB_03,1.611603,15.78346,23,5,Ready
93,2012.1.00108.S,B,uid://A001/X196/X39,NGC253_114.7_a_03_TP,ALMA_RB_03,0.7925405,-25.28819,6,2,Ready
111,2013.1.01365.S,B,uid://A002/X996c88/X83,W43-MM1_a_06_TP,ALMA_RB_06,18.79625,-1.908194,26,0,Ready
168,2013.1.00556.S,B,uid://A001/X197/Xc,LMC-N11B_115.3_a_03_TP,ALMA_RB_03,4.948246,-66.40623,12,2,Ready
188,2013.1.00191.S,A,uid://A001/X147/X204,NGC253_a_03_TP,ALMA_RB_03,0.7925372,-25.2888,19,0,Ready
193,2013.1.01329.S,B,uid://A001/X196/X11d,n613_a_03_TP,ALMA_RB_03,1.571732,-29.41849,2,0,Ready


In [58]:
alma1 = ephem.Observer()
alma1.lat = '-23.0262015'
alma1.long = '-67.7551257'
alma1.elev = 5060
alma1.horizon = ephem.degrees(str('30'))

In [59]:
uran = ephem.Uranus()
alma1.date = ephem.now() + 2
uran.compute(alma1)
print uran.ra, uran.dec
print uran.rise_time
print uran.transit_time
print uran.set_time

1:07:49.05 6:31:53.3
2015/5/6 11:08:52
2015/5/6 14:42:16
2015/5/6 18:15:39


In [60]:
obj = ephem.FixedBody()

In [61]:
#3c279
obj._ra = ephem.hours('12.93644')
obj._dec = ephem.degrees('-5.789313')
obj.compute(alma1)
alma1.date = ephem.now()
print alma1.date
print obj.ra, obj.dec
print obj.rise_time, obj.set_time

alma1.date = obj.rise_time
print alma1.date
print alma1.sidereal_time()

alma1.date = obj.set_time
print alma1.date
print alma1.sidereal_time()

2015/5/4 16:15:42
12:57:00.13 -5:52:24.8
2015/5/6 22:30:42 2015/5/6 06:32:17
2015/5/6 22:30:42
8:57:31.51
2015/5/6 06:32:17
16:56:28.73


In [62]:
#3c454
obj._ra = ephem.hours('22.89937')
obj._dec = ephem.degrees('16.14821')
obj.compute(alma1)
alma1.date = ephem.now()
print alma1.date
print obj.ra, obj.dec
print obj.rise_time, obj.transit_time, obj.set_time

alma1.date = obj.rise_time
print alma1.date
print alma1.sidereal_time()

alma1.date = obj.set_time
print alma1.date
print alma1.sidereal_time()

2015/5/4 16:15:42
22:54:42.85 16:13:41.1
2015/5/6 09:24:16 2015/5/6 12:29:42 2015/5/6 15:34:48
2015/5/6 09:24:16
19:48:55.60
2015/5/6 15:34:48
2:00:29.41


In [63]:
sun = ephem.Sun()
alma1.date = ephem.now()
sun.compute(alma1)
print sun.ra, sun.dec
print ephem.separation(uran, sun)

2:45:42.77 16:00:18.6
25:46:07.8


In [64]:
tm_amplitude_ready = TPlist.query('RA_y == 0').copy()

In [65]:
tm_amplitude_ready.sort(['PRJ_LETTER_GRADE'])

,CODE,PRJ_LETTER_GRADE,SB_UID,SG_ID,sbName,band,repfreq,RA_x,DEC_x,execount,observed,name,RA_y,DEC_y,solarSystem,SB_STATE
14,2013.1.01058.S,A,uid://A001/X148/Xac,uid://A001/X111/X21f_1,calibrator_uranus_07_TP,ALMA_RB_07,347.2,0,0,1,1,Amplitude,0,0,Uranus,Ready
189,2013.1.00191.S,A,uid://A001/X147/X205,uid://A001/X10e/X1e_2,Uranus_a_06_TP,ALMA_RB_06,230.3391,0,0,1,0,Amplitude,0,0,Uranus,Ready
0,2013.1.00652.S,B,uid://A002/X9a055b/X7e,uid://A001/X10e/X63b_1,Uranus_230.45_a_06_TP,ALMA_RB_06,230.4457,0,0,1,0,Amplitude,0,0,Uranus,Ready
2,2013.1.00652.S,B,uid://A002/X9a055b/X80,uid://A001/X10e/X63b_2,Uranus_230.45_b_06_TP,ALMA_RB_06,230.4457,0,0,1,0,Amplitude,0,0,Uranus,Ready
4,2013.1.00652.S,B,uid://A002/X9a055b/X82,uid://A001/X10e/X63b_3,Uranus_230.45_c_06_TP,ALMA_RB_06,230.4457,0,0,1,0,Amplitude,0,0,Uranus,Ready
6,2013.1.00652.S,B,uid://A002/X9a055b/X84,uid://A001/X10e/X63b_4,Uranus_230.42_d_06_TP,ALMA_RB_06,230.4227,0,0,1,0,Amplitude,0,0,Uranus,Ready
8,2012.1.00320.S,B,uid://A001/X147/Xf6,uid://A001/Xa0/X685_1,Ampcal_Uranus_a_06_TP,ALMA_RB_06,232.0294,0,0,1,0,Amplitude,0,0,Uranus,Ready
10,2013.1.00911.S,B,uid://A001/X147/Xcf,uid://A001/X110/X1d_2,Amplical_Uranus_a_07_TP,ALMA_RB_07,344.6617,0,0,1,0,Amplitude,0,0,Uranus,Ready
16,2012.1.00781.S,B,uid://A002/X9ae744/X1,uid://A001/Xa0/X1316_1,Uranus_a_07_TP,ALMA_RB_07,342.9355,0,0,1,0,Amplitude,0,0,Uranus,Ready
18,2012.1.00781.S,B,uid://A002/X9ae744/X3,uid://A001/Xa0/X1316_2,Uranus_b_07_TP,ALMA_RB_07,345.8491,0,0,1,0,Amplitude,0,0,Uranus,Ready


In [66]:
obser = datas.summary_sb.groupby('SG_ID').agg({'observed': np.sum})
started = obser.query('observed > 0').index.values

In [70]:
ok_ura_sg = tm_amplitude_ready.query('RA_y == 0 and SG_ID in @started').SG_ID.unique()
ok_ura_ampSb = tm_amplitude_ready.query('RA_y == 0').SB_UID.unique()
ok_ura_scSb = TPlist.query('SG_ID in @ok_ura_sg and SB_UID not in @ok_ura_ampSb and SB_STATE != "FullyObserved"')
ok_ura_scSb[['CODE', 'PRJ_LETTER_GRADE', 'SB_UID', 'sbName', 'band', 'RA_x', 'DEC_x', 'execount', 'observed', 'SB_STATE']].sort(['PRJ_LETTER_GRADE', 'CODE', 'observed'], ascending = [1,1,0])

,CODE,PRJ_LETTER_GRADE,SB_UID,sbName,band,RA_x,DEC_x,execount,observed,SB_STATE
190,2013.1.00191.S,A,uid://A001/X147/X206,NGC253_a_06_TP,ALMA_RB_06,0.7925372,-25.2888,10,1,Ready
15,2013.1.01058.S,A,uid://A001/X148/Xad,SgrA_sta_a_07_TP,ALMA_RB_07,17.76112,-29.00782,20,9,Ready
87,2012.1.00013.S,B,uid://A001/X148/X124,HH111MM_a_06_TP,ALMA_RB_06,5.862848,2.808236,13,0,Ready
29,2012.1.00133.S,B,uid://A002/X996c88/X59,G0.25+0._b_06_TP,ALMA_RB_06,17.769,-28.70991,6,0,Ready
196,2012.1.00178.S,B,uid://A002/X996c88/Xb,L1689N_a_07_TP,ALMA_RB_07,16.54144,-24.48239,8,0,Ready
198,2012.1.00178.S,B,uid://A002/X996c88/Xd,L1689N_b_07_TP,ALMA_RB_07,16.54144,-24.48239,12,0,Waiting
9,2012.1.00320.S,B,uid://A001/X147/Xf7,NGC_6302_a_06_TP,ALMA_RB_06,17.229,-37.10359,9,0,Ready
69,2012.1.00368.S,B,uid://A002/X9908b7/X24,Serpens__a_07_TP,ALMA_RB_07,18.49944,1.229167,4,0,Ready
71,2012.1.00368.S,B,uid://A002/X9908b7/X26,Serpens__a_06_TP,ALMA_RB_06,18.49944,1.229167,5,0,Ready
182,2012.1.00524.S,B,uid://A001/X147/Xa5,Mira_a_07_TP,ALMA_RB_07,2.322442,-2.977638,1,0,Ready
